In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Load and process dataset

In [2]:
import pandas as pd
import json
import os

# --- Configuration ---
REPO_BASE = "https://raw.githubusercontent.com/ArkadiusDS/MIPD/master/data"
OUTPUT_DIR = "/content/drive/MyDrive"

# Mapping input filenames (without extension) to output suffixes
# Key = CSV file name on GitHub, Value = Suffix for local JSONL file
# e.g., 'validation' reads 'validation.csv' and saves 'mipd_val.jsonl'
files_map = {
    "train": "train",
    "validation": "val",
    "test": "test"
}

# Global variable to store technique columns (detected from the training set)
TECHNIQUES_COLUMNS = []

def format_record(row, techniques_cols):
    """Przetwarza jeden wiersz DataFrame na format treningowy."""

    # 1. Wyciągnij aktywne techniki (gdzie wartość w kolumnie jest True/1)
    # Using the passed techniques_cols list to ensure consistency
    active_techniques = [col for col in techniques_cols if row[col]]

    # 2. Stwórz Target (Output) w formacie JSON z Markdown
    json_content = json.dumps({"discovered_techniques": active_techniques}, ensure_ascii=False)
    output_str = f"```json\n{json_content}\n```"

    # 3. Zwróć słownik w formacie dla Unsloth
    return {
        "input": row['article'],
        "output": output_str
    }

# --- Main Processing Loop ---

for input_name, output_suffix in files_map.items():

    # 1. Construct URL
    url = f"{REPO_BASE}/manipulation/{input_name}.csv"
    print(f"\n--- Processing: {input_name} ---")
    print(f"Downloading data from: {url}")

    try:
        # 2. Load Data
        df = pd.read_csv(url)
        print(f"Successfully loaded {len(df)} records.")

        # 3. Detect Columns (Only need to do this once, typically on the train set)
        # We assume the structure is identical across files.
        if not TECHNIQUES_COLUMNS:
            try:
                start_col = df.columns.get_loc('REFERENCE_ERROR')
                end_col = df.columns.get_loc('QUOTE_MINING') + 1
                TECHNIQUES_COLUMNS = df.columns[start_col:end_col].tolist()
                print(f"Defined manipulation techniques columns ({len(TECHNIQUES_COLUMNS)}): {TECHNIQUES_COLUMNS}")
            except KeyError as e:
                print(f"Critical Error: Could not find technique columns in {input_name}. Missing: {e}")
                continue

        # Optional: Print info for verification
        # df.info()

        # 4. Apply Formatting
        # We use a lambda to pass the specific columns list to the function
        print("Formatting records...")
        formatted_df = df.apply(
            lambda row: format_record(row, TECHNIQUES_COLUMNS),
            axis=1,
            result_type='expand'
        )

        # 5. Save to JSONL
        output_file = os.path.join(OUTPUT_DIR, f"mipd_{output_suffix}.jsonl")
        formatted_df.to_json(output_file, orient='records', lines=True, force_ascii=False)

        print(f"Saved {len(formatted_df)} records to: {output_file}")
        print("Sample record:", formatted_df.iloc[0].to_dict())

    except Exception as e:
        print(f"Error processing {input_name}: {e}")


--- Processing: train ---
Successfully loaded 10749 records.
Defined manipulation techniques columns (11): ['REFERENCE_ERROR', 'WHATABOUTISM', 'STRAWMAN', 'EMOTIONAL_CONTENT', 'CHERRY_PICKING', 'FALSE_CAUSE', 'MISLEADING_CLICKBAI', 'ANECDOTE', 'LEADING_QUESTIONS', 'EXAGGERATION', 'QUOTE_MINING']
Formatting records...
Saved 10749 records to: /content/drive/MyDrive/mipd_train.jsonl
Sample record: {'input': '"Terapia homoseksualizmu – szansa czy oszustwo? - Strona Życia" "Osoby te są przypuszczalnie sterowane przez środowiska gejowskie, ale też przez dziennikarzy, którzy szukają gorącego medialnego tematu. Np. redaktor Jan Józefowicz z telewizji WTK jako gej szukający pomocy przyszedł do poradni z ukrytym nadajnikiem, który obsługiwała będąca w pobliżu jego redakcyjna koleżanka. Jednocześnie jest przecież spora grupa osób, które naprawdę szukają pomocy w przezwyciężeniu własnego homoseksualizmu. Jak odróżnić jednych od drugich, jak chroniąc się przed prowokatorami – nie odmówić pomocy po

##Inspect processed records

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

first_record = formatted_df.iloc[2].to_dict()
print(f"Input:\n{first_record['input']}")
print(f"Output:\n{first_record['output']}")

Input:
"Delirium moŋe byæ wczesnym symptomem COVID-19 u starszych osób" "- Majaczenie lub dezorientacja u starszych osób mogą być wczesnym sygnałem COVID-19, nawet jeśli nie widzimy typowych objawów choroby - uważają naukowcy z Uniwersytetu Massachusetts. Wraz z grupą naukowców z kilku innych placówek medycznych w USA przeanalizowali symptomy COVID-19 pojawiające się u starszych hospitalizowanych pacjentów w wieku powyżej 65 lat. Wyniki badaņ opublikowało czasopismo naukowe JAMA Network Open. - Delirium jest powszechne wśród starszych pacjentów oddziałów ratunkowych (SOR) i wiąże się z ciężkim przebiegiem choroby i wyższą śmiertelnością. Często też pozostaje nierozpoznane - czytamy w artykule. - Majaczenie, dezorientacja, nieuwaga, splątanie oraz inne zaburzenia poznawcze są bowiem częstym objawem infekcji u starszych osób. Dzieje się tak dlatego, że układ odpornościowy ludzi w podeszłym wieku inaczej reaguje na infekcję, niż u młodych osób. Naukowcy zadali sobie pytanie, jak często za

##Filter out more than 16k tokens records from training dataset(GPU needed)

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel
from google.colab import userdata
import torch

max_seq_length = 16384
base_model_dir = "drive/MyDrive/bielik-4.5b-base"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_dir,
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    use_gradient_checkpointing = "unsloth",
)

NotImplementedError: Unsloth cannot find any torch accelerator? You need a GPU.

In [ ]:
def get_token_length(text):
    """Calculates token length using the loaded tokenizer."""
    if not isinstance(text, str):
        return 0
    return len(tokenizer.encode(text))

TOKEN_LIMIT = 16128

print(f"Filtering datasets to ensure combined 'instruction' + 'input' token length is less than {TOKEN_LIMIT} tokens, using the loaded tokenizer.")

In [ ]:
import pandas as pd

# Apply filtering to the training DataFrame
formatted_df['combined_token_length'] = formatted_df.apply(lambda row: get_token_length(row['instruction']) + get_token_length(row['input']), axis=1)
filtered_formatted_df = formatted_df[formatted_df['combined_token_length'] < TOKEN_LIMIT].copy()
print(f"Original formatted_df records: {len(formatted_df)}")
print(f"Filtered formatted_df records (token length < {TOKEN_LIMIT}): {len(filtered_formatted_df)}")

# Drop the temporary 'combined_token_length' column
filtered_formatted_df = filtered_formatted_df.drop(columns=['combined_token_length'])

# Save the filtered training dataset to a new JSONL file
output_file_16k = "/content/drive/MyDrive/mipd_train_16k.jsonl"
filtered_formatted_df.to_json(output_file_16k, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(filtered_formatted_df)} rekordów do pliku: {output_file_16k}")
print("Przykładowy rekord z przefiltrowanego zbioru:", filtered_formatted_df.iloc[0].to_dict())

##Create mini training set for mlops testing

In [3]:
import pandas as pd
import os

# Define paths
OUTPUT_DIR = "/content/drive/MyDrive"
input_file = os.path.join(OUTPUT_DIR, "mipd_train_cot_clean.jsonl")
output_file = os.path.join(OUTPUT_DIR, "mlops_testing_train_cot.jsonl")

try:
    # Load the full dataset
    full_df = pd.read_json(input_file, lines=True)
    print(f"Successfully loaded {len(full_df)} records from {input_file}")

    # Pick 30 random samples
    if len(full_df) >= 30:
        mini_df = full_df.sample(n=10, random_state=42) # Using random_state for reproducibility
        print(f"Selected 10 random samples.")
    else:
        mini_df = full_df.copy()
        print(f"Dataset has less than 10 records. Using all {len(mini_df)} records.")

    # Save the mini dataset
    mini_df.to_json(output_file, orient='records', lines=True, force_ascii=False)
    print(f"Saved {len(mini_df)} records to: {output_file}")

    # Display a sample record from the mini dataset
    if not mini_df.empty:
        print("Sample record from mlops_testing_train_cot:", mini_df.iloc[0].to_dict())

except FileNotFoundError:
    print(f"Error: The file {input_file} was not found. Please ensure it exists in your Google Drive.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully loaded 9903 records from /content/drive/MyDrive/mipd_train_cot_clean.jsonl
Selected 10 random samples.
Saved 10 records to: /content/drive/MyDrive/mlops_testing_train_cot.jsonl
Sample record from mlops_testing_train_cot: {'input': '"W Europie podpalane są maszty. Powodem teorie spiskowe na temat sieci 5G" "Teoria mówiąca o szkodliwości sieci 5G i rozprzestrzenianiu przez nią koronawirusa przyczyniła się do podpalenia dwóch masztów pod Amsterdamem - wynika z ustaleń tamtejszej policji. Do zdarzenia doszło w trakcie świąt wielkanocnych. Co ciekawe, zaatakowane maszty nie miały nadajnika 5G. – To bardzo niebezpieczne, niszczona jest ważna infrastruktura – ostrzegł Rob Bongenaar, dyrektor holenderskiego zrzeszenia telekomów Monet. Zwłaszcza że żadne badania nie wykazały związku między telefonią komórkową a obniżeniem odporności u człowieka, a fale radiowe nie tworzą ani nie rozprzestrzeniają wirusów. Dezinformacja na temat 5G Ogromnym powodzeniem w internecie cieszą się filmy,